<a href="https://colab.research.google.com/github/Knightler/deep-learning-practice/blob/main/PYTORCH_Experiment_Tracking_Project01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 PyTorch Experiment Tracking

In [ ]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.5.1+cu124
0.20.1+cu124
